In [1]:
import logging
import boto3
from botocore.exceptions import ClientError
from botocore.exceptions import NoCredentialsError
import os
import json
from datetime import datetime
import numpy as np
import pandas as pd
import re
import time

In [342]:
data = pd.read_excel("data/Spreadsheets/receipt_ground_truth.xlsx")
data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/Spreadsheets/receipt_ground_truth.xlsx'

In [113]:
# Getting rid of null values in date column
data['date']=data['date'].fillna(pd.Timestamp('1900-01-01'))

In [115]:
data.head()

,receipt_extract,object_name,date,subtotal,total,category
0,NaN,Airfare1.jpg,2024-09-22,45.00,45.00,Travel
1,NaN,Airfare2.jpg,2024-11-13,545.11,614.69,Travel
2,NaN,CarRental1.jpg,2024-12-15,355.00,505.63,Travel
3,NaN,CarRental2.jpg,1900-01-01,173.14,173.14,Travel
4,NaN,CarRental3.jpg,2024-12-20,272.83,319.18,Travel


In [117]:
# Uploading a new file to S3

def upload_file_to_s3(file_name, bucket_name, object_name=None):
    """
    Uploads a file to an S3 bucket.
    
    :param file_name: Path to the file to upload
    :param bucket_name: Name of the S3 bucket
    :param object_name: S3 object name. If not specified, file_name is used
    :return: a string of the response
    """
    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = file_name

    # Initialize the S3 client
    s3 = boto3.client('s3')

    
    try:
        with open(file_name, "rb") as file_data: # Uploading the FILE CONTENTS not the filepath
            response = s3.put_object(
                Body=file_data,
                Bucket=bucket_name,
                Key=object_name,                # This is the what the file will be called in S3
            )
        s = response
        return s
    except FileNotFoundError:
        print(f"The file {file_name} was not found.")
    except NoCredentialsError:
        print("AWS credentials not available.")

In [118]:
# Analyze a receipt in an S3 bucket

def analyze_receipt(bucket_name, object_name):
    """
    :param bucket_name: Name of the S3 bucket
    :param object_name: S3 object name
    :return: string of the response
    """
    
    client = boto3.client('textract')

    try:
        response = client.analyze_expense(
            
            Document = {
                "S3Object": {
                    "Bucket": bucket_name,
                    "Name": object_name
                }
            }
        )
        s = response
        return s
        
    except FileNotFoundError:
        print(f"The file {file_name} was not found.")
    except NoCredentialsError:
        print("AWS credentials not available.")

In [119]:
def condense_textract(text_extract, exclude = []):
    """
    Converts the large json response from Textract into a smaller dictionary.
    
    :param text_extract: json return of AWS Textract operation
    :param exclude: list of keys to exclude
    :return: the new dictionary
    """
    condensed_extract = {}

    # grabbing information from the text_extract json
    for i in range(len(text_extract['ExpenseDocuments'][0]['SummaryFields'])):
        key = text_extract['ExpenseDocuments'][0]['SummaryFields'][i]['Type']['Text']
        value = text_extract['ExpenseDocuments'][0]['SummaryFields'][i]['ValueDetection']['Text']
        if key not in exclude:
            if key not in condensed_extract.keys():
                condensed_extract[key] = value

            else:
                temp = " " + value
                condensed_extract[key] + temp
        
        if len(text_extract['ExpenseDocuments'][0]['LineItemGroups'][0]['LineItems'])> 0:
            condensed_extract['items'] = {}
            for j in range(len(text_extract['ExpenseDocuments'][0]['LineItemGroups'][0]['LineItems'][0]['LineItemExpenseFields'])):
                value = text_extract['ExpenseDocuments'][0]['LineItemGroups'][0]['LineItems'][0]['LineItemExpenseFields'][j]['ValueDetection']['Text']
                condensed_extract['items']['item'+str(j)] = value
    
    return condensed_extract

    

In [120]:
def post_to_s3_analyze_receipt(dataframe, bucket_name):
    
    for i in range(len(dataframe)):
        
        object_name = dataframe.loc[i, 'object_name']
        file_name = 'data/' + str(object_name)

        # Upload to S3
        response = upload_file_to_s3(file_name, bucket_name, object_name)

        # Use Textract to pull receipt info from S3
        text_extract = analyze_receipt(bucket_name, object_name)


        # condensing the text_extract into usable information
        condensed_extract = condense_textract(text_extract)
        
        dataframe.loc[i, 'receipt_extract'] = json.dumps(condensed_extract) # convert to string format for storage
        print(f"Index {i}, {object_name} finished")


In [121]:

dataframe = data.copy(deep = True)
bucket_name = bucket_name = "test-bucket-cnevares-2024"
post_to_s3_analyze_receipt(dataframe, bucket_name)

dataframe.to_excel('receipts_with_extracts.xlsx', index=False)  # Set index=False to exclude the index

print("DataFrame written to 'receipts_with_extracts.xlsx'")

C:\Users\Chris Nevares\AppData\Local\Temp\ipykernel_18840\2357253178.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '{"NAME": "DELTA", "INVOICE_RECEIPT_DATE": "22Sep24", "INVOICE_RECEIPT_ID": "0062265322160", "TOTAL": "USD45.00", "VENDOR_NAME": "DELTA"}' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  dataframe.loc[i, 'receipt_extract'] = json.dumps(condensed_extract) # convert to string format for storage


Index 0, Airfare1.jpg finished
Index 1, Airfare2.jpg finished
Index 2, CarRental1.jpg finished
Index 3, CarRental2.jpg finished
Index 4, CarRental3.jpg finished
Index 5, CarRental4.jpg finished
Index 6, CarRental5.jpg finished
Index 7, CarRental6.jpg finished
Index 8, CarWash1.jpg finished
Index 9, Gas1.jpg finished
Index 10, Gas2.jpg finished
Index 11, Gas3.jpg finished
Index 12, Gas4.jpg finished
Index 13, Groceries1.jpg finished
Index 14, Groceries2.jpg finished
Index 15, Groceries3.jpg finished
Index 16, Groceries4.jpg finished
Index 17, Groceries5.jpg finished
Index 18, Hotel1.jpg finished
Index 19, Hotel2.jpg finished
Index 20, Hotel3.jpg finished
Index 21, Hotel4.jpg finished
Index 22, Hotel5.jpg finished
Index 23, Hotel6.jpg finished
Index 24, Hotel7.jpg finished
Index 25, Hotel8.jpg finished
Index 26, Hotel9.jpg finished
Index 27, Meals1.jpg finished
Index 28, Meals2.jpg finished
Index 29, Meals3.jpg finished
Index 30, Meals4.jpg finished
Index 31, Meals5.jpg finished
Index 32

In [2]:
# Reloading dataframe from spreadsheet and appending extracted fields
receipts = pd.read_excel('data/Spreadsheets/receipts_with_extracts.xlsx')
receipts.head()

,receipt_extract,object_name,date,subtotal,total,category
0,"{""NAME"": ""DELTA"", ""INVOICE_RECEIPT_DATE"": ""22S...",Airfare1.jpg,2024-09-22,45.00,45.00,Travel
1,"{""NAME"": ""Alaska Airlines"", ""items"": {""item0"":...",Airfare2.jpg,2024-11-13,545.11,614.69,Travel
2,"{""NAME"": ""National."", ""items"": {""item0"": ""TIME...",CarRental1.jpg,2024-12-15,355.00,505.63,Travel
3,"{""AMOUNT_PAID"": ""$0.00"", ""items"": {""item0"": ""C...",CarRental2.jpg,1900-01-01,173.14,173.14,Travel
4,"{""ADDRESS"": ""BOZEMAN INTL ARPT\n850 GALLATIN F...",CarRental3.jpg,2024-12-20,272.83,319.18,Travel


In [344]:
# reformatting date column to datetime m/d/yyyy
receipts['date'] = pd.to_datetime(receipts['date'])
receipts['date'] = receipts['date'].dt.strftime('%m/%d/%Y')

In [345]:
# The fields we are looking for from the receipt_extract dictionary are INVOICE_RECEIPT_DATE and TOTAL

In [78]:
# Simple function to extract an amount from a string input from Textract

def extract_amt_from_string(s):
    regex = r'\d+(\.\d{2})?'
    match = re.search(regex, s)
    
    if match:
        amount = np.float64(match.group())
        return amount.round(2)
        
    else:
        return np.float64(0.00).round(2)

In [79]:
# Simple function to extract a date from a string input from Textract

def extract_date_from_string(s):
    regex = r'\b\d{1,2}[-/]\d{1,2}[-/]\d{2}\d{2}?\b'
    dates = re.findall(regex, s)
    
    if len(dates) > 1:
        return dates[-1]
    elif len(dates) == 1:
        return dates[0]
    
    return ""

In [80]:
# We'll use this to convert whatever date Textract retrieved into a datetime object format m/d/yyyy.

def reformat_date(date_string):
    # List of potential input formats
    input_formats = ["%m/%d/%y", "%m/%d/%Y", "%m/%-d/%y", "%m/%-d/%Y", "%-m/%d/%y", "%-m/%d/%Y", "%B %d %Y", 
                     "%b %d %Y", '%a %b %d', '%d%b%y', '%d-%b-%Y', '%m/%d', "%Y-%m-%d", "%m-%d", '%m/%d/%y'
    ]
    
    # Try parsing with each format
    for fmt in input_formats:
        try:
            date_object = datetime.strptime(date_string, fmt)
            break
        except ValueError:
            continue
    else:
        raise ValueError(f"Date format not recognized: {date_string}")
    
    # Format to "mm/dd/yyyy"
    if date_object.year == 1900:
        date_object = date_object.replace(year = 2024)
        
    date_object = date_object.strftime("%m/%d/%Y")

        
    return date_object #return date portion of datetime object

In [90]:
# Add the extracted values into our dataframe

def extract_date_amount(df):
    for i in range(len(df)):
        jason = json.loads(df.loc[i, 'receipt_extract'])

        if 'INVOICE_RECEIPT_DATE' in jason.keys():
            date = jason['INVOICE_RECEIPT_DATE'].replace(',',' ').replace('.', ' ').strip()
            reformatted_date = reformat_date(date)
            
        else:
            # If field not found, attempt to extract from text
            date = extract_date_from_string(df.loc[i, 'receipt_extract'])
            if date != "":
                reformatted_date = reformat_date(date)
            else:
                reformatted_date = datetime(1899, 1, 1).strftime('%m/%d/%Y')
        
        df.loc[i, 'extracted_date'] = reformatted_date
        
        if 'TOTAL' in jason.keys() and extract_amt_from_string(jason['TOTAL']) != 0.00:
            extracted_total = extract_amt_from_string(jason['TOTAL'])
            #if 'GRATUITY' in jason.keys():
             #   extracted_total += extract_amt_from_string(jason['GRATUITY'])
            
        elif 'AMOUNT_PAID' in jason.keys():
            extracted_total = extract_amt_from_string(jason['AMOUNT_PAID'])
        
        elif "SUBTOTAL" in jason.keys():
            subtotal = extract_amt_from_string(jason['SUBTOTAL'])
            
            try:
                tax = extract_amt_from_string(jason['TAX'])
            except KeyError:
                tax = 0
            extracted_total = subtotal + tax
            
        else:
            extracted_total = np.float64(0.00)
            
        df.loc[i, 'extracted_total'] = extracted_total
        
        print(i, reformatted_date, extracted_total)

In [91]:
extract_date_amount(receipts)

0 09/22/2024 45.0
1 11/14/2024 614.69
2 12/15/2024 505.63
3 01/01/1899 173.14
4 01/01/1899 319.18
5 01/01/1899 210.19
6 01/01/1899 294.25
7 01/01/1899 102.54
8 12/20/2024 24.0
9 12/20/2024 22.63
10 12/18/2024 35.63
11 12/18/2024 14.54
12 11/14/2024 48.73
13 12/05/2024 38.46
14 09/24/2024 22.48
15 10/22/2024 21.76
16 10/16/2024 22.74
17 11/13/2024 1.99
18 12/20/2024 514.01
19 01/01/1899 444.09
20 10/08/2024 209.0
21 01/01/1899 634.88
22 01/01/1899 182.9
23 01/01/1899 805.56
24 01/01/1899 461.38
25 12/15/2024 0.0
26 01/01/1899 42.0
27 10/30/2024 11.49
28 12/16/2024 37.37
29 12/15/2024 15.05
30 12/14/2024 22.63
31 04/23/2024 14.37
32 03/12/2024 13.3
33 10/23/2024 35.55
34 09/30/2024 43.6
35 09/12/2024 29.92
36 09/04/2024 52.82
37 12/21/2024 392.0
38 12/21/2024 119.79
39 10/26/2024 607.8
40 12/19/2024 7.25
41 11/19/2024 1994.21
42 12/23/2024 0.0
43 12/19/2024 111.0
44 12/12/2024 20.0
45 08/13/2024 165.29
46 09/10/2024 0.0
47 10/18/2024 322.67
48 12/18/2024 119.96
49 12/17/2024 79.28
50 10/

In [92]:
receipts

,receipt_extract,object_name,date,subtotal,total,category,extracted_date,extracted_total,predicted_category
0,"{""NAME"": ""DELTA"", ""INVOICE_RECEIPT_DATE"": ""22S...",Airfare1.jpg,09/22/2024,45.00,45.00,Travel,09/22/2024,45.00,Meals
1,"{""NAME"": ""Alaska Airlines"", ""items"": {""item0"":...",Airfare2.jpg,11/13/2024,545.11,614.69,Travel,11/14/2024,614.69,Travel
2,"{""NAME"": ""National."", ""items"": {""item0"": ""TIME...",CarRental1.jpg,12/15/2024,355.00,505.63,Travel,12/15/2024,505.63,Travel
3,"{""AMOUNT_PAID"": ""$0.00"", ""items"": {""item0"": ""C...",CarRental2.jpg,01/01/1900,173.14,173.14,Travel,01/01/1899,173.14,Travel
4,"{""ADDRESS"": ""BOZEMAN INTL ARPT\n850 GALLATIN F...",CarRental3.jpg,12/20/2024,272.83,319.18,Travel,01/01/1899,319.18,Travel
5,"{""ADDRESS"": ""ANCHORAGE INTL AIRPORT\n(ANC)\n50...",CarRental4.jpg,12/19/2024,143.60,210.19,Travel,01/01/1899,210.19,Travel
6,"{""ADDRESS"": ""Budget I 379 Interpace Parkway I ...",CarRental5.jpg,01/01/1900,213.60,294.25,Travel,01/01/1899,294.25,Travel
7,"{""ADDRESS"": ""BUDGET RENT-A-CAR OF FAIRBANKS\nP...",CarRental6.jpg,11/13/2024,80.00,102.54,Travel,01/01/1899,102.54,Travel
8,"{""ADDRESS"": ""Rapid Clean Bozeman\n2191 Burke S...",CarWash1.jpg,12/20/2024,24.00,24.00,Travel,12/20/2024,24.00,Meals
9,"{""ADDRESS"": ""Holiday St 2746506\n402 Belgrade ...",Gas1.jpg,12/20/2024,22.63,22.63,Travel,12/20/2024,22.63,Meals


In [93]:
def convert_dict_to_string_with_prompt(dictionary):
    
    prompt = '''You are an expert in receipt categorization. Categorize the following receipt into one of these categories: Meals, Supplies, Safety, Travel, Lodging, or Other. 
    Respond in the format: "Category:<category>"
    Do not include explanations, steps, or any additional text.
    Receipt:

    '''
    for key in dictionary.keys():
        if key == 'items':
            prompt+=key +":\n"
            for k in dictionary['items'].keys():
                prompt+= k + ":" + dictionary['items'][k].replace('\n',' ') +'\n'
        else:
            prompt += key +":"+dictionary[key]+"\n"
    prompt+="Category:"

    return prompt

In [14]:
def prompt_model_titan_express(json_derulo):
    client = boto3.client('bedrock-runtime')
    try:
        response = client.invoke_model(
            modelId = 'amazon.titan-text-lite-v1',
            contentType = 'application/json',
            accept = "application/json",
            body = json.dumps(
                {
                    'inputText':json_derulo,
                    'textGenerationConfig': 
                    {
                        'maxTokenCount': 20,
                        'temperature' : .5,
                        'topP':.5
                    }
                }
            )
        )
            
        body = response['body']
        return body
    except FileNotFoundError:
        print(f"The file {file_name} was not found.")
    except NoCredentialsError:
        print("AWS credentials not available.")

In [11]:
def add_category_to_dataframe(dataframe, start = 0, end = 1):

    for i in range(start, end):
        receipt_extract = json.loads(dataframe.loc[i, 'receipt_extract'])

        prompt = convert_dict_to_string_with_prompt(receipt_extract)

        response = prompt_model_titan_express(prompt)

        # Converts response into dictionary format
        parsed_body = json.loads(response.read().decode('utf-8'))

        # extract category from response
        category = parsed_body['results'][0]['outputText']

        dataframe.loc[i, 'predicted_category'] = category
        print(i, category, dataframe.loc[i, 'category'])
        time.sleep(5)

In [355]:
add_category_to_dataframe(receipts, 0, 25)

0  Meals Travel
1 Travel Travel
2  Travel Travel
3  Travel Travel
4 Travel Travel
5  Travel Travel
6  Travel Travel
7 Travel Travel
8  Meals Travel
9 Meals Travel
10 Supplies Travel
11 Supplies Travel
12 Travel Travel
13 Meals Meals
14 Meals Meals
15 Meals Meals
16  Other Meals
17  Meals Meals
18  Lodging Lodging
19  Lodging Lodging
20  Lodging Lodging
21  Lodging Lodging
22  Lodging Lodging
23  Lodging Lodging
24  Lodging Lodging


In [22]:
# Format strings correctly. There are sometimes leading spaces.
receipts['predicted_category'] = receipts['predicted_category'].str.strip()

In [99]:
# Reloading and running the rest of the receipts 
receipts = pd.read_csv("data/Spreadsheets/receipts_with_text_extracts.csv")

In [15]:
add_category_to_dataframe(receipts, 25, len(receipts))

25  Lodging Lodging
26  Lodging Lodging
27 Meals Meals
28 Meals Meals
29 Meals Meals
30 Meals Meals
31 Meals Meals
32 Meals Meals
33 Meals Meals
34 Meals Meals
35 Meals Meals
36 Meals Meals
37 Meals Meals
38 Meals Meals
39  Meals Meals
40  Meals Meals
41 Other Other
42  Lodging Travel
43  Meals Travel
44 Lodging Travel
45  Supplies Supplies
46  Other Supplies
47 Supplies Supplies
48  Supplies Supplies
49 Supplies Supplies
50  Other Supplies
51 Supplies Supplies
52 Supplies Supplies
53  Supplies Supplies
54 Meals Travel
55 Travel Travel
56 Meals Travel
57  Meals Travel
58 Meals Travel


In [55]:
def calculate_accuracies(df):
    category_correct = 0
    date_correct = 0
    amount_correct = 0
    category_miss = []
    date_miss = []
    amount_miss = []
    total = len(df)
    for i in range(total):
        if df.loc[i, 'category'] == df.loc[i, 'predicted_category']:
            category_correct+=1
        else:
            category_miss.append(i)
        
        if df.loc[i, 'date'] == df.loc[i, 'extracted_date']:
            date_correct+=1
        else:
            date_miss.append(i)

        if df.loc[i, 'total'] == df.loc[i, 'extracted_total']:
            amount_correct+=1
        else:
            amount_miss.append(i)

    print(f'Accuracies:\nAmount: {(amount_correct/total)*100:.2f}\nDate: {(date_correct/total)*100:.2f}\nCategory: {(category_correct/total)*100:.2f}')
    print(f'Misses:\nAmount: {amount_miss}\nDate: {date_miss}\nCategory{category_miss}')

In [100]:
calculate_accuracies(receipts)

Accuracies:
Amount: 84.75
Date: 67.80
Category: 74.58
Misses:
Amount: [25, 26, 30, 33, 34, 35, 37, 42, 46]
Date: [1, 3, 4, 5, 6, 7, 19, 20, 21, 22, 23, 24, 25, 26, 42, 47, 53, 57, 58]
Category[0, 8, 9, 10, 11, 16, 42, 43, 44, 46, 50, 54, 56, 57, 58]


In [95]:
# Checking only Meals type expenses
meals_receipts = receipts[receipts['category']=='Meals']
meals_receipts = meals_receipts.reset_index(drop=True)
meals_receipts

,receipt_extract,object_name,date,subtotal,total,category,extracted_date,extracted_total,predicted_category
0,"{""ADDRESS"": ""LAUREL #1287\n312 FIRST AVE S\nLA...",Groceries1.jpg,12/05/2024,38.46,38.46,Meals,12/05/2024,38.46,Meals
1,"{""ADDRESS"": ""Rockin J\n2993 Highway 78\nAbsaro...",Groceries2.jpg,09/24/2024,22.48,22.48,Meals,09/24/2024,22.48,Meals
2,"{""ADDRESS"": ""COLUMBUS #1151\n602 8TH AVE NORTH...",Groceries3.jpg,10/22/2024,21.76,21.76,Meals,10/22/2024,21.76,Meals
3,"{""ADDRESS"": ""ESSENTIAL\nFUELS #5\n2646\nUS-310...",Groceries4.jpg,10/16/2024,22.74,22.74,Meals,10/16/2024,22.74,Other
4,"{""NAME"": ""EXCHANGE"", ""items"": {""item0"": ""61276...",Groceries5.jpg,11/13/2024,1.99,1.99,Meals,11/13/2024,1.99,Meals
5,"{""ADDRESS"": ""McDonald's\nRestaurant\n#11906\n9...",Meals1.jpg,10/30/2024,11.49,11.49,Meals,10/30/2024,11.49,Meals
6,"{""NAME"": ""Chipotle Mexican Grill"", ""items"": {""...",Meals2.jpg,12/16/2024,25.10,37.37,Meals,12/16/2024,37.37,Meals
7,"{""ADDRESS"": ""Peet's Coffee & Tea\n1901 Webster...",Meals3.jpg,12/15/2024,15.05,15.05,Meals,12/15/2024,15.05,Meals
8,"{""ADDRESS"": ""CORRALES MEXICAN FOOD\n7951 THOMP...",Meals4.jpg,12/14/2024,20.25,26.70,Meals,12/14/2024,22.63,Meals
9,"{""ADDRESS"": ""Welcome to Chick-fil-A\nSouthcent...",Meals5.jpg,04/23/2024,13.04,14.37,Meals,04/23/2024,14.37,Meals


In [96]:
calculate_accuracies(meals_receipts)

Accuracies:
Amount: 73.68
Date: 100.00
Category: 94.74
Misses:
Amount: [8, 11, 12, 13, 15]
Date: []
Category[3]


In [97]:
# Write new spreadsheet to a file
receipts.to_csv('data/Spreadsheets/receipts_with_text_extracts.csv', index=False)  # Set index=False to exclude the index

print("DataFrame written to 'receipts_with_text_extracts.csv'")

DataFrame written to 'receipts_with_text_extracts.csv'


In [98]:
meals_receipts.loc[11, 'receipt_extract']

'{"ADDRESS": "F Street Station\\n325 F Street\\nAK 99501\\nAnchorage", "items": {"item0": "Domestic", "item1": "2", "item2": "11.00", "item3": "Description\\n2 11.00\\nDomestic\\nI 23.00"}, "STREET": "325 F Street", "CITY": "Anchorage", "STATE": "AK", "ZIP_CODE": "99501", "NAME": "F Street Station", "ADDRESS_BLOCK": "325 F Street\\nAK 99501\\nAnchorage", "AMOUNT_PAID": "35.55", "GRATUITY": "", "INVOICE_RECEIPT_DATE": "10/23/2024", "INVOICE_RECEIPT_ID": "2024-10 7311/1", "SUBTOTAL": "35.00", "TAX": "0.55", "TOTAL": "35.55", "VENDOR_ADDRESS": "F Street Station\\n325 F Street\\nAK 99501\\nAnchorage", "VENDOR_NAME": "F Street Station", "VENDOR_PHONE": "(907) 272-5196", "OTHER": "33"}'